In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202540.csv")

In [3]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)

ema_def["acumulado_combinado"] = (
    ema_def.groupby("store_name")["acumulado_combinado"]
    .transform(lambda x: x / x.max())
).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [4]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord = ema_ord[~(ema_ord["store_name"]=="0")]

In [5]:
ema_ord["acumulado_combinado"].describe()

count    430080.000000
mean          0.977999
std           0.089474
min           0.020000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: acumulado_combinado, dtype: float64

In [6]:
ema_ord[(ema_ord["store_name"]=="SUCURSAL NORTE") & (ema_ord["product_ref"]=="DAE02286025")]

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
283883,SUCURSAL NORTE,DAE02286025,2025.0,38,1.661153,97279.53,161596.18,0.0008,0.2250,1.444445e+10,89386.09,0.49,0.0,2989365.61,0.0,0.34,AAA
283884,SUCURSAL NORTE,DAE02286025,2025.0,39,1.328922,97279.53,129276.91,0.0006,0.2596,1.021288e+05,0.79,0.54,10.0,5671435.27,1.0,0.61,A
283885,SUCURSAL NORTE,DAE02286025,2025.0,40,1.063138,97279.53,103421.56,0.0005,0.3061,1.023873e+05,0.99,0.60,11.0,7742307.79,1.0,0.66,A
283886,SUCURSAL NORTE,DAE02286025,2025.0,41,0.850510,97279.53,82737.21,0.0004,0.3596,9.101090e+04,1.10,0.65,12.0,9835624.05,1.0,0.70,A
283887,SUCURSAL NORTE,DAE02286025,2025.0,42,0.680408,97279.53,66189.77,0.0003,0.4143,7.678010e+04,1.16,0.69,13.0,13246814.33,1.0,0.74,A
283888,SUCURSAL NORTE,DAE02286025,2025.0,43,0.544326,97279.53,52951.78,0.0002,0.4776,6.407170e+04,1.21,0.75,14.0,16757081.50,1.0,0.80,B
283889,SUCURSAL NORTE,DAE02286025,2025.0,44,0.435461,97279.53,42361.44,0.0002,0.5324,5.252820e+04,1.24,0.82,15.0,20382393.13,1.0,0.81,B
283874,SUCURSAL NORTE,DAE02286025,2025.0,29,0.320000,97279.53,31129.45,0.0001,0.6327,0.000000e+00,0.00,0.90,0.0,27754918.64,0.0,0.91,B
283875,SUCURSAL NORTE,DAE02286025,2025.0,30,0.288000,97279.53,28016.50,0.0001,0.6553,0.000000e+00,0.00,0.90,0.0,30350112.53,0.0,0.92,B
283876,SUCURSAL NORTE,DAE02286025,2025.0,31,0.259200,97279.53,25214.85,0.0001,0.6792,5.705470e+08,22627.42,0.90,0.0,32916125.81,0.0,0.92,B


## NIVEL DE SERVICIO

In [7]:
# Definimos las condiciones para la meta, basadas en la clasificación
condiciones_meta = [
    (ema_ord['Clasificacion'] == 'AAA'),
    (ema_ord['Clasificacion'] == 'A'),
    (ema_ord['Clasificacion'] == 'B'),
    (ema_ord['Clasificacion'] == 'C')
]

# Definimos los valores de la meta que se asignarán a cada clasificación
metas = [0.999, 0.95, 0.90, 0.50]

ema_ord = ema_ord.copy()

# Asignamos la nueva columna 'Meta' usando np.select
ema_ord.loc[:, 'Nivel de servicio'] = np.select(condiciones_meta, metas, default=np.nan)

In [8]:
ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
205002,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,32,64.531200,13771.02,888660.45,0.0009,0.0996,195505.3,0.22,0.00,1.272727,0.22,0.5,0.15,AAA,0.999
205003,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,33,57.824960,13771.02,796308.68,0.0008,0.1039,175187.9,0.22,0.00,2.881890,0.44,0.8,0.22,AAA,0.999
205011,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,41,55.394413,13771.02,762837.57,0.0008,0.1103,152567.5,0.20,0.01,4.059150,0.64,1.0,0.28,AAA,0.999
205004,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,34,55.059968,13771.02,758231.92,0.0008,0.1119,181975.7,0.24,0.01,3.210526,0.88,1.0,0.28,AAA,0.999
247019,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,41,51.828099,10188.65,528058.36,0.0006,0.1868,110892.3,0.21,0.01,3.990566,1.09,1.0,0.30,AAA,0.999


In [9]:
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [10]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
PRINCIPAL COTA,1578,407,1691,50084
SUCURSAL BARRANQUILLA,1905,650,2099,49106
SUCURSAL BUCARAMANGA,2063,658,2219,48820
SUCURSAL CALI,1813,527,2149,49271
SUCURSAL CALLE 6,1824,749,2209,48978
SUCURSAL MEDELLIN,1511,519,1911,49819
SUCURSAL NORTE,1656,493,2063,49548
SUCURSAL VALLADOLID,1430,440,1906,49984


In [11]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df_real =df.copy()


In [12]:
df_mio.head()

,store_name,product_ref,semana,Ema,acumulado_combinado,Clasificacion
71648,PRINCIPAL COTA,BCE00606125,32,27.358720,0.02,AAA
71649,PRINCIPAL COTA,BCE00606125,33,21.886976,0.25,AAA
399630,PRINCIPAL COTA,DCS10536137,42,21.416133,0.23,AAA
399631,PRINCIPAL COTA,DCS10536137,43,19.274520,0.23,AAA
399619,PRINCIPAL COTA,DCS10536137,31,18.083800,0.20,AAA


In [13]:
df_real = df_real[["store_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]

In [14]:
df_real

,store_name,product_name,semana,ema,acumulado_combinado,clasificacion
0,SUCURSAL BARRANQUILLA,DAB02570025,43,41.083909,0.256684,AAA
1,SUCURSAL BARRANQUILLA,DAB14570025,43,36.201236,0.302723,AAA
2,SUCURSAL BARRANQUILLA,BLS00037125,43,28.525297,0.218257,AAA
3,SUCURSAL BARRANQUILLA,BCS00035125,43,27.018937,0.244767,AAA
4,SUCURSAL BARRANQUILLA,DAC00037189,43,26.558686,0.401244,AAA
...,...,...,...,...,...,...
43852,SUCURSAL NORTE,DAE05466003,43,0.000000,1.000000,C
43853,SUCURSAL NORTE,DAE05094003,43,0.000000,1.000000,C
43854,SUCURSAL NORTE,DAE06511003,43,0.000000,1.000000,C
43855,SUCURSAL NORTE,DAE06627002,43,0.000000,1.000000,C
